# **Crawlers - Machado de Assis**

Nessa página, são dois crawlers:

1. É o crawler para teste, com um único arquivo. Para validar testes nos processos subsequentes com pouco conteúdo. Ele baixa um único PDF de Machado de Assis no Domínio público, salvando-a na pasta "machadoteste" do drive.

2. O segundo crawler segundo busca em todas as páginas do link e baixa todos os pdfs da obra de Machado de Assis no Domínio Público, salvando-os na pasta "machado" do drive.

**1. Crawler de arquivo único para teste**

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Install necessary libraries
!pip install selenium beautifulsoup4 aws-lambda-powertools requests jmespath
!pip install chromedriver_autoinstaller


In [ ]:
import chromedriver_autoinstaller
import time
from abc import ABC, abstractmethod
from tempfile import mkdtemp
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from aws_lambda_powertools import Logger

logger = Logger(service="decodingml/crawler")

# Instalar automaticamente a versão correta do ChromeDriver
chromedriver_autoinstaller.install()

class BaseDocument:
    def __init__(self, platform, content, link, author_id):
        self.platform = platform
        self.content = content
        self.link = link
        self.author_id = author_id

    def save(self):
        # Simulação de salvamento no Google Drive
        file_name = self.link.split("/")[-1]
        save_path = f"/content/drive/My Drive/machado1/{file_name}"
        with open(save_path, "wb") as f:
            f.write(self.content)
        logger.info(f"Saved PDF: {save_path}")

class BaseCrawler(ABC):
    model: type[BaseDocument]

    @abstractmethod
    def extract(self, link: str, **kwargs) -> None: ...

class BaseAbstractCrawler(BaseCrawler, ABC):
    def __init__(self, scroll_limit: int = 5) -> None:
        options = Options()
        options.binary_location = "/usr/bin/google-chrome"  # Caminho do Chrome no Colab
        options.add_argument("--no-sandbox")
        options.add_argument("--headless=new")
        options.add_argument("--single-process")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--log-level=3")
        options.add_argument("--disable-popup-blocking")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-dev-tools")
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--no-zygote")
        options.add_argument(f"--user-data-dir={mkdtemp()}")
        options.add_argument(f"--data-path={mkdtemp()}")
        options.add_argument(f"--disk-cache-dir={mkdtemp()}")
        options.add_argument("--remote-debugging-port=9222")

        self.set_extra_driver_options(options)

        self.scroll_limit = scroll_limit
        self.driver = webdriver.Chrome(
            options=options,
        )

    def set_extra_driver_options(self, options: Options) -> None:
        pass

    def login(self) -> None:
        pass

    def scroll_page(self) -> None:
        """Scroll through the page based on the scroll limit."""
        current_scroll = 0
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        while True:
            self.driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);"
            )
            time.sleep(5)
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height or (
                self.scroll_limit and current_scroll >= self.scroll_limit
            ):
                break
            last_height = new_height
            current_scroll += 1

class PdfCrawler(BaseAbstractCrawler):
    model = BaseDocument  # Substitua com o modelo adequado

    def set_extra_driver_options(self, options) -> None:
        options.add_argument(r"--profile-directory=Profile 2")

    def extract(self, link: str, **kwargs) -> None:
        logger.info(f"Starting scraping PDF links: {link}")

        self.driver.get(link)
        self.scroll_page()

        soup = BeautifulSoup(self.driver.page_source, "html.parser")

        # Procurar pela imagem do botão "Baixar" e encontrar o elemento pai "a"
        download_image = soup.find("img", {"src": "../img/botoes/baixar.gif"})
        if download_image:
            download_button = download_image.find_parent("a")
            if download_button:
                pdf_url = download_button["href"]
                if pdf_url.startswith(".."):
                    pdf_url = "http://www.dominiopublico.gov.br" + pdf_url[2:]
                elif pdf_url.startswith("/"):
                    pdf_url = "http://www.dominiopublico.gov.br" + pdf_url
                else:
                    pdf_url = "http://www.dominiopublico.gov.br/pesquisa/" + pdf_url
                print(f"Found PDF URL: {pdf_url}")
                logger.info(f"Found PDF URL: {pdf_url}")

                pdf_content = self.download_pdf(pdf_url)
                if pdf_content:
                    self.save_pdf(pdf_url, pdf_content, kwargs.get("user"))
            else:
                logger.info("No parent link found for download image.")
        else:
            logger.info("No download image found.")

        logger.info(f"Successfully scraped and saved PDFs from: {link}")
        self.driver.close()

    def download_pdf(self, pdf_url: str) -> bytes:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            print(f"Successfully downloaded PDF: {pdf_url}")  # Confirmação de download
            return response.content
        else:
            logger.error(f"Failed to download PDF: {pdf_url}")
            return None

    def save_pdf(self, pdf_url: str, pdf_content: bytes, user_id: str) -> None:
        instance = self.model(
            platform="dominiopublico",
            content=pdf_content,
            link=pdf_url,
            author_id=user_id
        )
        instance.save()

    def login(self):
        pass  # No login required for this site

# Exemplo de uso:
crawler = PdfCrawler()
crawler.extract(
    "http://www.dominiopublico.gov.br/pesquisa/DetalheObraForm.do?select_action=&co_obra=2039",
    user="user_id_example"
)


{"level":"INFO","location":"extract:96","message":"Starting scraping PDF links: http://www.dominiopublico.gov.br/pesquisa/DetalheObraForm.do?select_action=&co_obra=2039","timestamp":"2024-07-20 16:32:18,390+0000","service":"decodingml/crawler"}
Found PDF URL: http://www.dominiopublico.gov.br/pesquisa/DetalheObraDownload.do?select_action=&co_obra=2039&co_midia=2
{"level":"INFO","location":"extract:116","message":"Found PDF URL: http://www.dominiopublico.gov.br/pesquisa/DetalheObraDownload.do?select_action=&co_obra=2039&co_midia=2","timestamp":"2024-07-20 16:32:24,493+0000","service":"decodingml/crawler"}
Successfully downloaded PDF: http://www.dominiopublico.gov.br/pesquisa/DetalheObraDownload.do?select_action=&co_obra=2039&co_midia=2
{"level":"INFO","location":"save:31","message":"Saved PDF: /content/drive/My Drive/machadoteste/DetalheObraDownload.do?select_action=&co_obra=2039&co_midia=2","timestamp":"2024-07-20 16:32:26,241+0000","service":"decodingml/crawler"}
{"level":"INFO","locat

**1. Crawler de todos os PDFs de Machado de Assis no Domínio Público.**

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Install necessary libraries
!pip install selenium beautifulsoup4 aws-lambda-powertools requests jmespath
!pip install chromedriver_autoinstaller


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2024-07-20 15:34:03--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.125.91, 142.250.125.93, 142.250.125.190, ...
Connecting to dl.google.com (dl.google.com)|142.250.125.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108772160 (104M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.5’

google-chrome-stabl 100%[===================>] 103.73M   315MB/s    in 0.3s    

2024-07-20 15:34:04 (315 MB/s) - ‘google-chrome-stable_current_amd64.deb.5’ saved [108772160/108772160]

(Reading database ... 123864 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (126.0.6478.182-1) over (126.0.6478.182-1) ...
Setting up google-chrome-stable (126.0.6

In [ ]:
import time
from abc import ABC, abstractmethod
from tempfile import mkdtemp
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from aws_lambda_powertools import Logger
import chromedriver_autoinstaller

logger = Logger(service="decodingml/crawler")

class BaseDocument:
    def __init__(self, platform, content, link, author_id):
        self.platform = platform
        self.content = content
        self.link = link
        self.author_id = author_id

    def save(self):
        # Simulação de salvamento no Google Drive
        file_name = self.link.split("/")[-1]
        save_path = f"/content/drive/My Drive/machado/{file_name}"
        with open(save_path, "wb") as f:
            f.write(self.content)
        logger.info(f"Saved PDF: {save_path}")

class BaseCrawler(ABC):
    model: type[BaseDocument]

    @abstractmethod
    def extract(self, link: str, **kwargs) -> None: ...

class BaseAbstractCrawler(BaseCrawler, ABC):
    def __init__(self, scroll_limit: int = 5) -> None:
        options = Options()
        options.binary_location = "/usr/bin/google-chrome"  # Caminho do Chrome no Colab
        options.add_argument("--no-sandbox")
        options.add_argument("--headless=new")
        options.add_argument("--single-process")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--log-level=3")
        options.add_argument("--disable-popup-blocking")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-dev-tools")
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--no-zygote")
        options.add_argument(f"--user-data-dir={mkdtemp()}")
        options.add_argument(f"--data-path={mkdtemp()}")
        options.add_argument(f"--disk-cache-dir={mkdtemp()}")
        options.add_argument("--remote-debugging-port=9222")

        self.set_extra_driver_options(options)

        chromedriver_autoinstaller.install()

        self.scroll_limit = scroll_limit
        self.driver = webdriver.Chrome(options=options)

    def set_extra_driver_options(self, options: Options) -> None:
        pass

    def login(self) -> None:
        pass

    def scroll_page(self) -> None:
        """Scroll through the page based on the scroll limit."""
        current_scroll = 0
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        while True:
            self.driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);"
            )
            time.sleep(5)
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height or (
                self.scroll_limit and current_scroll >= self.scroll_limit
            ):
                break
            last_height = new_height
            current_scroll += 1

class PdfCrawler(BaseAbstractCrawler):
    model = BaseDocument  # Substitua com o modelo adequado

    def set_extra_driver_options(self, options) -> None:
        options.add_argument(r"--profile-directory=Profile 2")

    def extract(self, link: str, **kwargs) -> None:
        logger.info(f"Starting scraping PDF links: {link}")

        # Iterar sobre as 7 páginas
        for page_number in range(1, 8):
            page_link = f"http://www.dominiopublico.gov.br/pesquisa/ResultadoPesquisaObraForm.do?first={(page_number-1)*50}&skip={(page_number-1)*50}&ds_titulo=&co_autor=&no_autor=Machado%20de%20Assis&co_categoria=2&pagina={page_number}&select_action=Submit&co_midia=2&co_obra=&co_idioma=&colunaOrdenar=null&ordem=null"
            logger.info(f"Scraping page: {page_link}")

            self.driver.get(page_link)
            self.scroll_page()

            soup = BeautifulSoup(self.driver.page_source, "html.parser")

            # Procurar por todos os links que conduzem para páginas de detalhes das obras
            detail_links = soup.find_all("a", href=True, target="_blank", style="cursor:hand")
            detail_urls = ["http://www.dominiopublico.gov.br" + link["href"][2:] for link in detail_links]

            for detail_url in detail_urls:
                self.scrape_detail_page(detail_url, kwargs.get("user"))

        logger.info(f"Successfully scraped and saved PDFs from: {link}")
        self.driver.close()

    def scrape_detail_page(self, link: str, user_id: str) -> None:
        logger.info(f"Scraping detail page: {link}")

        self.driver.get(link)
        self.scroll_page()

        soup = BeautifulSoup(self.driver.page_source, "html.parser")

        # Procurar pela imagem do botão "Baixar" e encontrar o elemento pai "a"
        download_image = soup.find("img", {"src": "../img/botoes/baixar.gif"})
        if download_image:
            download_button = download_image.find_parent("a")
            if download_button:
                pdf_url = download_button["href"]
                if pdf_url.startswith(".."):
                    pdf_url = "http://www.dominiopublico.gov.br" + pdf_url[2:]
                elif pdf_url.startswith("/"):
                    pdf_url = "http://www.dominiopublico.gov.br" + pdf_url
                else:
                    pdf_url = "http://www.dominiopublico.gov.br/pesquisa/" + pdf_url
                print(f"Found PDF URL: {pdf_url}")
                logger.info(f"Found PDF URL: {pdf_url}")

                pdf_content = self.download_pdf(pdf_url)
                if pdf_content:
                    self.save_pdf(pdf_url, pdf_content, user_id)
            else:
                logger.info("No parent link found for download image.")
        else:
            logger.info("No download image found.")

    def download_pdf(self, pdf_url: str) -> bytes:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            print(f"Successfully downloaded PDF: {pdf_url}")  # Confirmação de download
            return response.content
        else:
            logger.error(f"Failed to download PDF: {pdf_url}")
            return None

    def save_pdf(self, pdf_url: str, pdf_content: bytes, user_id: str) -> None:
        instance = self.model(
            platform="dominiopublico",
            content=pdf_content,
            link=pdf_url,
            author_id=user_id
        )
        instance.save()

    def login(self):
        pass  # No login required for this site

# Exemplo de uso:
crawler = PdfCrawler()
crawler.extract(
    "http://www.dominiopublico.gov.br/pesquisa/ResultadoPesquisaObraForm.do?first=50&skip=0&ds_titulo=&co_autor=&no_autor=Machado%20de%20Assis&co_categoria=2&pagina=1&select_action=Submit&co_midia=2&co_obra=&co_idioma=&colunaOrdenar=null&ordem=null",
    user="user_id_example"
)


{"level":"INFO","location":"extract:93","message":"Starting scraping PDF links: http://www.dominiopublico.gov.br/pesquisa/ResultadoPesquisaObraForm.do?first=50&skip=0&ds_titulo=&co_autor=&no_autor=Machado%20de%20Assis&co_categoria=2&pagina=1&select_action=Submit&co_midia=2&co_obra=&co_idioma=&colunaOrdenar=null&ordem=null","timestamp":"2024-07-20 15:34:39,713+0000","service":"decodingml/crawler"}
{"level":"INFO","location":"extract:98","message":"Scraping page: http://www.dominiopublico.gov.br/pesquisa/ResultadoPesquisaObraForm.do?first=0&skip=0&ds_titulo=&co_autor=&no_autor=Machado%20de%20Assis&co_categoria=2&pagina=1&select_action=Submit&co_midia=2&co_obra=&co_idioma=&colunaOrdenar=null&ordem=null","timestamp":"2024-07-20 15:34:39,720+0000","service":"decodingml/crawler"}
{"level":"INFO","location":"extract:98","message":"Scraping page: http://www.dominiopublico.gov.br/pesquisa/ResultadoPesquisaObraForm.do?first=50&skip=50&ds_titulo=&co_autor=&no_autor=Machado%20de%20Assis&co_categor